In [ ]:
import pandas as pd
import json
from fyers_apiv3 import fyersModel
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

client_id = "UZJ1B1WF4N-100"
secret_key = "KJ7CW6C2I8"
redirect_uri = "https://www.google.com/"
response_type = "code"  
state = "sample_state"
grant_type = "authorization_code"  

session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key,
    redirect_uri=redirect_uri,
    response_type=response_type
)

response = session.generate_authcode()
response

In [ ]:

auth_code = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE3NDI3NDA1NTEsImV4cCI6MTc0Mjc3MDU1MSwibmJmIjoxNzQyNzM5OTUxLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJZSzA4OTczIiwib21zIjoiSzEiLCJoc21fa2V5IjoiZjRiNzM1ZmVjMTdkYmUwZjBiZDE1N2E0YzQ2ZjA1M2ZlZDg1OWIzZTc4N2YyOTM2MjBjZGI4NWIiLCJpc0RkcGlFbmFibGVkIjoiWSIsImlzTXRmRW5hYmxlZCI6Ik4iLCJub25jZSI6IiIsImFwcF9pZCI6IlVaSjFCMVdGNE4iLCJ1dWlkIjoiMjUzNmMyNThlOTY1NGFjOGEzYjFmMzNjMWIzMzg4MDgiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.E_-y6spBcgSo3HpikvwCXrR8C88SdBnM9j--1BmcJl8"

session = fyersModel.SessionModel(
    client_id=client_id,
    secret_key=secret_key, 
    redirect_uri=redirect_uri, 
    response_type=response_type, 
    grant_type=grant_type
)

session.set_token(auth_code)

response = session.generate_token()

access_token = response['access_token']

fyers = fyersModel.FyersModel(client_id=client_id, token=access_token)


In [ ]:
bank_nifty_stocks = [
    # "NSE:AUBANK-EQ", 
    # "NSE:AXISBANK-EQ", 
    # "NSE:BANKBARODA-EQ", 
    # "NSE:BANDHANBNK-EQ", 
    # "NSE:CANBK-EQ", 
    # "NSE:FEDERALBNK-EQ", 
    # "NSE:HDFCBANK-EQ", 
    # "NSE:ICICIBANK-EQ", 
    # "NSE:IDFCFIRSTB-EQ", 
    # "NSE:INDUSINDBK-EQ", 
    # "NSE:KOTAKBANK-EQ", 
    # "NSE:PNB-EQ", 
    # "NSE:SBIN-EQ", 
    # "NSE:NIFTYBANK-INDEX",
    "NSE:NIFTY50-INDEX"
]

def generate_date_ranges(start_date, end_date):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    date_ranges = []
    
    current_start = start_date
    while current_start < end_date:
        current_end = current_start + relativedelta(months=3) - timedelta(days=1)
        
        if current_end > end_date:
            current_end = end_date
        
        date_ranges.append({
            "range_from": current_start.strftime("%Y-%m-%d"),
            "range_to": current_end.strftime("%Y-%m-%d")
        })
        
        current_start = current_end + timedelta(days=1)
    
    return date_ranges

In [ ]:
ranges = generate_date_ranges("2020-01-01", "2025-03-22")
df = None
for i in range(0, len(bank_nifty_stocks)):

    symbol = bank_nifty_stocks[i]

    main_df = pd.DataFrame()

    for r in ranges:

        range_from = r['range_from']
        range_to = r['range_to']

        data = {
            "symbol":symbol,
            "resolution":"D",
            "date_format":"1", 
            "range_from":range_from, 
            "range_to":range_to, 
            "cont_flag":"1"
        }

        response = fyers.history(data=data)
        df = pd.DataFrame(response['candles'], columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='s', utc=True).dt.tz_convert('Asia/Kolkata')
        df['symbol'] = symbol.replace("NSE:", "").replace("-EQ", "").replace("-INDEX", "")
        df = df[['symbol', 'datetime', 'open', 'high', 'low', 'close', 'volume']]
        df['datetime'] = df['datetime'].dt.tz_localize(None)

        main_df = pd.concat([main_df, df])

    symbol = symbol.replace("NSE:", "").replace("-EQ", "").replace("-INDEX", "")
    print(symbol)

    main_df.to_csv(os.path.join("Banknifty_data_for_option", f"{symbol}.csv"), index=False)
